In [1]:
import pprint

# DarshanUtils for Python

This notebook gives an overwiew of features provided by the Python bindings for DarshanUtils.

By default only metadata, available modules and the name records are loaded:

In [2]:
import darshan
report = darshan.DarshanReport("example.darshan", read_all=True)  # Default behavior
report.info()

Filename:       example.darshan
Times:          2017-03-20 10:07:47 to 2017-03-20 10:09:43 (Duration 0:01:56)
Executeable:    /global/project/projectdirs/m888/glock/tokio-abc-results/bin.edison/vpicio_uni /scratch2/scratchdirs/glock/tokioabc-s.4478544/vpicio/vpicio.hdf5 32
Processes:      2048
JobID:          4478544
UID:            69615
Modules in Log: ['POSIX', 'MPI-IO', 'LUSTRE', 'STDIO']
Loaded Records: {'POSIX': 1, 'MPI-IO': 1, 'STDIO': 129}
Name Records:   4
Darshan/Hints:  {'lib_ver': '3.1.3', 'h': 'romio_no_indep_rw=true;cb_nodes=4'}
DarshanReport:  id(140181962385952) (tmp)


In [3]:
# report.metadata         # dictionary with raw metadata from darshan log
# report.modules          # dictionary with raw module info from darshan log (need: technical, module idx)
# report.name_records     # dictionary for resovling name records: id -> path/name
# report.records          # per module "dataframes"/dictionaries

The darshan report holds a variety of namespaces for report related data. All of them are also referenced in `report.data` at the moment, but reliance on this internal organization of the report object is discouraged once the API stabilized. Currently, `report.data` references the following information:

In [4]:
report.data.keys()

dict_keys(['version', 'metadata', 'records', 'summary', 'modules', 'counters', 'name_records', 'mounts'])

### Loading Additional Records

For memory efficiant analysis, it is possible to supress records from being loaded automatically. This is useful, for example, when analysis only considers a particular layer.

In [5]:
report = darshan.DarshanReport("example.darshan", read_all=False) # Loads no records!

In [6]:
# expected to fail, as no records were loaded
try:
    report.records['POSIX']
except:
    print("No POSIX records loaded for this report yet.")

No POSIX records loaded for this report yet.


Additional records then can be loaded selectively, for example, on a per module basis:

In [7]:
report.mod_read_all_records("POSIX") 

In [8]:
report.records['POSIX']

[{'id': 6301063301082038805,
  'rank': -1,
  'counters': array([                2049, 18446744073709551615, 18446744073709551615,
                            0,                16402,                16404,
                            0,                    0,                    0,
                            0, 18446744073709551615, 18446744073709551615,
                            0,                    0,                    0,
                2199023259968,                    0,        2199023261831,
                            0,                    0,                    0,
                        16384,                    0,                    0,
                            8,                16401,              1048576,
                            0,            134217728,                    0,
                            0,                    0,                    0,
                            0,                    0,                    0,
                            0,               

### Aggregation and Filtering (Experimental)

Darshan log data is routinely aggregated for quick overview. The report object offers a few methods to perform common aggregations:

Report aggregations and summarization remains **experimental** for now, mostly to allow interfaces to stabilize. But experimental features can be switched on easily by invoking `darshan.enable_experimental()`:

In [9]:
import darshan
darshan.enable_experimental(verbose=True) # Enable verbosity, listing new functionality

Added method create_time_summary to DarshanReport.
Added method print_module_records to DarshanReport.
Added method summarize to DarshanReport.
Added method merge to DarshanReport.
Added method create_timeline to DarshanReport.
Added method records_as_dict to DarshanReport.
Added method reduce to DarshanReport.
Added method agg_ioops to DarshanReport.
Added method create_sankey to DarshanReport.
Added method filter to DarshanReport.
Added method mod_agg_iohist to DarshanReport.
Added method name_records_summary to DarshanReport.


In [10]:
report.name_records_summary()

{6301063301082038805: {'name': '/scratch2/scratchdirs/glock/tokioabc-s.4478544/vpicio/vpicio.hdf5',
  'counts': {'POSIX': 1}}}

#### Chain operations like filtering and reductions
The filter and reduce operations return DarshanReports themsleves, thus allow to convieniently chain operations.

In [12]:
import darshan
darshan.enable_experimental()
report = darshan.DarshanReport("example.darshan", read_all=True)
report.name_records

{14734109647742566553: '<STDIN>',
 15920181672442173319: '<STDOUT>',
 7238257241479193519: '<STDERR>',
 6301063301082038805: '/scratch2/scratchdirs/glock/tokioabc-s.4478544/vpicio/vpicio.hdf5'}

In [13]:
report.filter(name_records=[6301063301082038805, 15920181672442173319]).records

{'POSIX': [{'id': 6301063301082038805,
   'rank': -1,
   'counters': array([                2049, 18446744073709551615, 18446744073709551615,
                             0,                16402,                16404,
                             0,                    0,                    0,
                             0, 18446744073709551615, 18446744073709551615,
                             0,                    0,                    0,
                 2199023259968,                    0,        2199023261831,
                             0,                    0,                    0,
                         16384,                    0,                    0,
                             8,                16401,              1048576,
                             0,            134217728,                    0,
                             0,                    0,                    0,
                             0,                    0,                    0,
                      

In [15]:
# reduce all after filtering
report.filter(pattern="*.hdf5").records

NO (.*?).hdf5 <STDIN>
NO (.*?).hdf5 <STDOUT>
NO (.*?).hdf5 <STDERR>
YES (.*?).hdf5 /scratch2/scratchdirs/glock/tokioabc-s.4478544/vpicio/vpicio.hdf5


{'POSIX': [{'id': 6301063301082038805,
   'rank': -1,
   'counters': array([                2049, 18446744073709551615, 18446744073709551615,
                             0,                16402,                16404,
                             0,                    0,                    0,
                             0, 18446744073709551615, 18446744073709551615,
                             0,                    0,                    0,
                 2199023259968,                    0,        2199023261831,
                             0,                    0,                    0,
                         16384,                    0,                    0,
                             8,                16401,              1048576,
                             0,            134217728,                    0,
                             0,                    0,                    0,
                             0,                    0,                    0,
                      

In [11]:
# only preserve some
report.filter(name_records=[6301063301082038805]).reduce(mods=['POSIX', 'STDIO']).records

{'POSIX': [{'id': '*',
   'rank': -1,
   'counters': array([                2049, 18446744073709551615, 18446744073709551615,
                             0,                16402,                16404,
                             0,                    0,                    0,
                             0, 18446744073709551615, 18446744073709551615,
                             0,                    0,                    0,
                 2199023259968,                    0,        2199023261831,
                             0,                    0,                    0,
                         16384,                    0,                    0,
                             8,                16401,              1048576,
                             0,            134217728,                    0,
                             0,                    0,                    0,
                             0,                    0,                    0,
                             0,       

In [16]:
# expected to fail
try:
    pprint.pprint(report.summary['agg_ioops'])
except:
    print("IOOPS have not been aggregated for this report.")

IOOPS have not been aggregated for this report.


In [17]:
report.read_all() 
report.summarize()

In [18]:
# attaches among other summarizes the aggregated ioops
try:
    pprint.pprint(report.summary['agg_ioops'])
except:
    print("IOOPS have not been aggregated for this report.")

{'MPI-IO': {'MPIIO_ACCESS1_ACCESS': 134217728,
            'MPIIO_ACCESS1_COUNT': 16384,
            'MPIIO_ACCESS2_ACCESS': 272,
            'MPIIO_ACCESS2_COUNT': 8,
            'MPIIO_ACCESS3_ACCESS': 544,
            'MPIIO_ACCESS3_COUNT': 2,
            'MPIIO_ACCESS4_ACCESS': 328,
            'MPIIO_ACCESS4_COUNT': 2,
            'MPIIO_BYTES_READ': 0,
            'MPIIO_BYTES_WRITTEN': 2199023259968,
            'MPIIO_COLL_OPENS': 2048,
            'MPIIO_COLL_READS': 0,
            'MPIIO_COLL_WRITES': 16384,
            'MPIIO_FASTEST_RANK': 597,
            'MPIIO_FASTEST_RANK_BYTES': 1073741824,
            'MPIIO_HINTS': 0,
            'MPIIO_INDEP_OPENS': 0,
            'MPIIO_INDEP_READS': 0,
            'MPIIO_INDEP_WRITES': 18,
            'MPIIO_MAX_READ_TIME_SIZE': 0,
            'MPIIO_MAX_WRITE_TIME_SIZE': 134217728,
            'MPIIO_MODE': 9,
            'MPIIO_NB_READS': 0,
            'MPIIO_NB_WRITES': 0,
            'MPIIO_RW_SWITCHES': 0,
            'MPIIO

Or fine grained:

In [19]:
report.mod_agg_iohist("MPI-IO")  # to create the histograms

{'READ_0_100': 0,
 'READ_100_1K': 0,
 'READ_1K_10K': 0,
 'READ_10K_100K': 0,
 'READ_100K_1M': 0,
 'READ_1M_4M': 0,
 'READ_4M_10M': 0,
 'READ_10M_100M': 0,
 'READ_100M_1G': 0,
 'READ_1G_PLUS': 0,
 'WRITE_0_100': 4,
 'WRITE_100_1K': 14,
 'WRITE_1K_10K': 0,
 'WRITE_10K_100K': 0,
 'WRITE_100K_1M': 0,
 'WRITE_1M_4M': 0,
 'WRITE_4M_10M': 0,
 'WRITE_10M_100M': 0,
 'WRITE_100M_1G': 16384,
 'WRITE_1G_PLUS': 0}

In [20]:
report.agg_ioops()               # to create the combined operation type summary

{'MPI-IO': {'MPIIO_INDEP_OPENS': 0,
  'MPIIO_COLL_OPENS': 2048,
  'MPIIO_INDEP_READS': 0,
  'MPIIO_INDEP_WRITES': 18,
  'MPIIO_COLL_READS': 0,
  'MPIIO_COLL_WRITES': 16384,
  'MPIIO_SPLIT_READS': 0,
  'MPIIO_SPLIT_WRITES': 0,
  'MPIIO_NB_READS': 0,
  'MPIIO_NB_WRITES': 0,
  'MPIIO_SYNCS': 0,
  'MPIIO_HINTS': 0,
  'MPIIO_VIEWS': 32768,
  'MPIIO_MODE': 9,
  'MPIIO_BYTES_READ': 0,
  'MPIIO_BYTES_WRITTEN': 2199023259968,
  'MPIIO_RW_SWITCHES': 0,
  'MPIIO_MAX_READ_TIME_SIZE': 0,
  'MPIIO_MAX_WRITE_TIME_SIZE': 134217728,
  'MPIIO_SIZE_READ_AGG_0_100': 0,
  'MPIIO_SIZE_READ_AGG_100_1K': 0,
  'MPIIO_SIZE_READ_AGG_1K_10K': 0,
  'MPIIO_SIZE_READ_AGG_10K_100K': 0,
  'MPIIO_SIZE_READ_AGG_100K_1M': 0,
  'MPIIO_SIZE_READ_AGG_1M_4M': 0,
  'MPIIO_SIZE_READ_AGG_4M_10M': 0,
  'MPIIO_SIZE_READ_AGG_10M_100M': 0,
  'MPIIO_SIZE_READ_AGG_100M_1G': 0,
  'MPIIO_SIZE_READ_AGG_1G_PLUS': 0,
  'MPIIO_SIZE_WRITE_AGG_0_100': 4,
  'MPIIO_SIZE_WRITE_AGG_100_1K': 14,
  'MPIIO_SIZE_WRITE_AGG_1K_10K': 0,
  'MPIIO_SIZE_W

### Report Algebra

Various operations are implemented to merge, combine and manipulate log records. This is useful for analysis task, but can also be used to construct performance projections or extrapolation.

For convienience, we overload some of the operations provided by Python when they resemble intuitive equivalence to their mathematical counterparts. In particular, we enable the combination of different object types.

In [21]:
# merging records
r1 = darshan.DarshanReport("example.darshan", read_all=True)
r2 = darshan.DarshanReport("example2.darshan", read_all=True)
rx = r1 + r2
plot_opcounts(rx)

NameError: name 'plot_opcounts' is not defined

In [ ]:
# multiply records with a scalar (think, four times the I/O load)
r1 = darshan.DarshanReport("example.darshan", read_all=True)
rx = r1 * 4
plot_opcounts(rx)

In [ ]:
# rebase via timedelta
r1 = darshan.DarshanReport("example.darshan", read_all=True)
dt = datetime.timedelta()
rx = r1 + dt


## Plotting

In [ ]:
import darshan
darshan.enable_experimental(verbose=False)

r3 = darshan.DarshanReport("example.darshan")
r3.mod_read_all_records('POSIX')

from darshan.experimental.plots.matplotlib import plot_access_histogram
plot_access_histogram(r3, mod='POSIX')

In [ ]:
import darshan
darshan.enable_experimental(verbose=False)

r3 = darshan.DarshanReport("example.darshan")
r3.mod_read_all_records('MPI-IO')

from darshan.experimental.plots.matplotlib import plot_access_histogram
plot_access_histogram(r3, mod='MPI-IO')

In [ ]:
import darshan
darshan.enable_experimental(verbose=False)

r3 = darshan.DarshanReport("example.darshan")
r3.read_all()

from darshan.experimental.plots.matplotlib import plot_opcounts
plot_opcounts(r3)

### DXT Records

DXT records are also supported, and can be loaded individually on a per module basis as follows:


In [ ]:
report2 = darshan.DarshanReport("example.darshan")
report2.mod_read_all_dxt_records("ABC") 

In [ ]:
report2.data

## Exporting Data for Use in Third-Party Analysis

Darshan logs may be used in contexts beyond our imagination. To make this effortless export in JSON is easy.

In [ ]:
import darshan
report = darshan.DarshanReport("example.darshan", read_all=True)
report.to_json()

## Error Handling?

Currently, playing with two modes, both have their pros and cons.

Generally, should expose errors and let users handle them. At the same time, just skipping invalid load requests does little harm but greatly improves convienince.

Could add a switch to enable disable these guard :/

In [ ]:
report = darshan.DarshanReport("example.darshan")

In [ ]:
report.mod_read_all_records("ABC") # Expect KeyError

In [ ]:
report.mod_read_all_dxt_records("ABC") # Expect printed warning, but not exception